In [ ]:
from ipydatagrid import DataGrid, Filter, Sort

In [ ]:
from json import load

with open('./cars.json') as fobj:  
    data = load(fobj)

In [ ]:
datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100)
datagrid

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])